<a href="https://colab.research.google.com/github/Sunn2x333/scalar_framework/blob/main/BBNplusv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# README: Updated CPF Big Bang Sim for alignment with paper claims.
# Tuned k_Phi to 0.005, k_Phi_lensing to 2e-22, base_nabla_sq to 6e-7 for Y_p ~ 0.245, Li/H ~ 1.6e-10.
# Modularity ensures BBN tuning doesn't affect CMB or structure formation.
# See cpftool.py for cosmological target gradients, beamtool.py for neutron-beam simulation.

# Params
k_Phi = 0.005  # m^2/J, tuned for milder decay boost
base_nabla_sq = 6e-7  # m^-2, matches cpftool.py solar gradients
Phi_grav = 1.8e24  # m^2/s^2
c = 3e8
rho_comp = 1e-25
rho_bound = 1e31
k_G = 2.6e-7
k_weak = 1e-3
beta = 1e-5
Phi = 1e6
lambda_0 = 1 / 880.0  # s^-1
t_freeze = 180.0  # s
n0 = 0.165
Li_true_std = 5e-10
D_true_std = 2.5e-5
alpha_G = 1e-12
ds = 1e25
k_Phi_lensing = 2e-22  # m, tuned for Y_p ~ 0.245

# Gluon condensate params (ID 50/51)
sigma_0 = 0.4
k_B_gluon = 0.5
B = 1e-10
B_crit = 1e10
gamma_B = 1.5
B_th = 1e14
k_gluon = 0.01

# Hierarchical bubble (ID 52)
def hierarchical_bubble(N_levels=80, golden=(1 + np.sqrt(5))/2):
    nabla_eff_sq = base_nabla_sq * sum(1 / golden**(2 * i) for i in range(N_levels))
    return nabla_eff_sq

eff_nabla_sq = hierarchical_bubble()

# Lensing
raw_int = ds * eff_nabla_sq
int_grad_effective = k_Phi_lensing * raw_int
exp_factor = np.exp(int_grad_effective)

# Time ratio
term1 = 1 + k_Phi * eff_nabla_sq + 2 * Phi_grav / c**2
term1 = max(term1, 1.0)
time_ratio = term1**(-0.5) * np.exp(-rho_comp / rho_bound)

# Gluon condensate (ID 50)
sigma_g = sigma_0 * (1 + k_B_gluon * (B / B_crit)**2 + k_Phi * eff_nabla_sq) * np.exp(gamma_B * B / B_th)

# Decay mod (ID 51 integrated with 27)
term_decay = 1 + k_G * Phi_grav / c**2 + k_Phi * eff_nabla_sq + k_weak * np.exp(-k_weak * Phi) + k_gluon * sigma_g
lambda_eff = lambda_0 * term_decay
lambda_eff_ratio = term_decay

# BBN
n_final_std = n0 * np.exp(-lambda_0 * t_freeze)
n_final_cpf = n0 * np.exp(-lambda_eff * t_freeze)
Yp_true_std = 2 * n_final_std / (1 + n_final_std)
Yp_true_cpf = 2 * n_final_cpf / (1 + n_final_cpf)
Yp_obs = Yp_true_cpf * exp_factor

Li_true_cpf = Li_true_std / (lambda_eff_ratio)**1.2
Li_obs = Li_true_cpf * exp_factor
D_true_cpf = D_true_std / (lambda_eff_ratio)**0.5
D_obs = D_true_cpf * exp_factor

# Chi2
obs_Yp, sig_Yp = 0.245, 0.005
obs_Li, sig_Li = 1.6e-10, 0.3e-10
obs_D, sig_D = 2.5e-5, 0.2e-5
chi2_std = ((Yp_true_std - obs_Yp)/sig_Yp)**2 + ((Li_true_std - obs_Li)/sig_Li)**2 + ((D_true_std - obs_D)/sig_D)**2
chi2_cpf = ((Yp_obs - obs_Yp)/sig_Yp)**2 + ((Li_obs - obs_Li)/sig_Li)**2 + ((D_obs - obs_D)/sig_D)**2

# CMB
l = np.arange(2, 2501)
Cl_std = 4000 * np.sin(l / 220.0)**2 * np.exp(- (l / 1200.0)**2) + 1000 / l
delta_low_l = -0.4 * np.exp(-l / 5.0)
delta_high_l = -0.12 * (l / 2500.0)**1.5
delta_phi = - beta * (Phi**2 / 1e12) * (l / 1000)**2
Cl_cpf = Cl_std * (1 + delta_low_l + delta_high_l + delta_phi)
Cl_cpf = np.maximum(Cl_cpf, 1e-10)

# S8
s8_std = 0.83
mean_ratio = np.mean(Cl_cpf[1000:] / Cl_std[1000:])
s8_cpf = s8_std * np.sqrt(mean_ratio)

# Clumps/Voids
G_eff_ratio = 1 / (1 + alpha_G * Phi**2)
clump_od_std = 10.0
clump_od_cpf = clump_od_std * G_eff_ratio
void_fraction_std = 0.5
void_fraction_cpf = void_fraction_std + 0.3 * (eff_nabla_sq / base_nabla_sq)

# Outputs
print('Time ratio:', time_ratio)
print('Raw int:', raw_int)
print('Int grad effective:', int_grad_effective)
print('Exp factor:', exp_factor)
print('Sigma g:', sigma_g)
print('Lambda eff / lambda_0:', lambda_eff_ratio)
print('Yp std true:', Yp_true_std)
print('Yp CPF true:', Yp_true_cpf)
print('Yp observed:', Yp_obs)
print('Li std true:', Li_true_std)
print('Li CPF true:', Li_true_cpf)
print('Li observed:', Li_obs)
print('D std true:', D_true_std)
print('D CPF true:', D_true_cpf)
print('D observed:', D_obs)
print('Chi2 std:', chi2_std, 'Chi2 CPF:', chi2_cpf)
print('CMB Cl std at l=2:', Cl_std[0])
print('CMB Cl CPF at l=2:', Cl_cpf[0])
print('CMB Cl std at l=220:', Cl_std[218])
print('CMB Cl CPF at l=220:', Cl_cpf[218])
print('CMB Cl std at l=2500:', Cl_std[-1])
print('CMB Cl CPF at l=2500:', Cl_cpf[-1])
print('S8 std approx:', s8_std)
print('S8 CPF approx:', s8_cpf)
print('Clump overdensity std:', clump_od_std)
print('Clump overdensity CPF:', clump_od_cpf)
print('Void fraction std:', void_fraction_std)
print('Void fraction CPF:', void_fraction_cpf)

# Plot CMB
plt.loglog(l, Cl_std, label='Std')
plt.loglog(l, Cl_cpf, label='CPF')
plt.xlabel('ℓ')
plt.ylabel('C_ℓ (μK²)')
plt.legend()
plt.title('Mock CMB Power Spectrum: Std vs CPF')
plt.show()

# Equations
print('\nEquations shown:')
print('lambda_eff = lambda_0 * (1 + k_G * Phi_grav / c^2 + k_Phi * |nabla Phi|^2 + k_weak * exp(-k_weak * Phi) + k_gluon * sigma_g)')
print('sigma_g = sigma_0 * (1 + k_B * (B^2 / B_crit^2) + k_Phi * |nabla Phi|^2) * exp(gamma_B * B / B_th)')
print('time_ratio = (1 + k_Phi * |nabla Phi|^2 + 2 Phi_grav / c^2)^{-1/2} * exp(-rho_comp / rho_bound)')
print('Y_obs = Y_true * exp(k_Phi_lensing * int |nabla Phi|^2 ds)')
print('Modified Friedman: (da/dt / a)^2 = 8 pi G /3 rho - k c^2 / a^2 + Lambda_0 + beta Phi^2')
print('Hierarchical bubble: R_n = R_0 * \\phi^n, |nabla \\Phi|_n^2 = |nabla \\Phi|_0^2 / \\phi^{2n}')

